# Voronoi

> Fill in a module description here

In [ ]:
#| default_exp tutorials/Voronoi

#| export
## Introduction

Lets see if we can make some terrain

#| export
## Getting Started

In [ ]:
#| export
#data
from collections import namedtuple, defaultdict
from dataclasses import dataclass,  field, asdict
from typing import List
from enum import Enum

#math
from random import randint
import numpy as np

from fastcore.basics import patch

In [ ]:
#| export
from HexMagic.primitives import Hex, MapCord , HexPosition,  MapSize, MapRect, MapCord, MapPath ,HexGrid, HexRegion


In [ ]:
#| export
from HexMagic.styles import StyleCSS,  SVGBuilder

In [ ]:
#| export
def sampleGrid(hexDim = 2, radius=40, fill = "white",makeLabels = False):

    #drawing set up
    hexStyle = StyleCSS("HexStyle",fill=fill,stroke="blue",stroke_width=2)
    labelStyle = StyleCSS("labelStyle",fill=fill,stroke="black",stroke_width=1)
    
    grid = HexGrid.centered(hexDim,radius=radius,style=hexStyle)
    #grid.adjustRadius(radius)

    #Need to add styles as we go along
    grid.builder.add_style(labelStyle)

    #itterate through the hexes
    for i in range(len(grid.hexes)):
        if makeLabels:
            grid.hexes[i].label = str(i) 
        grid.hexes[i].labelStyle = labelStyle.name

    grid.update()
    return grid
    

grid = sampleGrid(makeLabels=True)
grid.builder.show()

In [ ]:
#| export
def demo_voronoi_growth_simple():
    """Simpler Voronoi growth using HexRegion list"""
    grid = sampleGrid(5, fill="lightgray")
    
    colors = [
        StyleCSS("region0", fill="#ff6b6b", stroke="darkred", stroke_width=0),
        StyleCSS("region1", fill="#4ecdc4", stroke="darkblue", stroke_width=0),
        StyleCSS("region2", fill="#95e1d3", stroke="darkgreen", stroke_width=0),
        StyleCSS("region3", fill="#f38181", stroke="darkorange", stroke_width=0),
    ]
    
    perimeter_style = StyleCSS("perimeter_path", fill="none", stroke="#333", stroke_width=3)
    arrow_style = StyleCSS("growth_arrow", stroke="#333333", stroke_width=2, fill="none")
    boundary_layer = ""
    for style in colors + [perimeter_style, arrow_style]:
        grid.builder.add_style(style)
    
    seeds = [
        grid.nCols * 1 + 1,
        grid.nCols * 1 + (grid.nCols - 2),
        grid.nCols * (grid.nRows - 2) + 1,
        grid.nCols * (grid.nRows - 2) + (grid.nCols - 2),
    ]
    
    regions = [HexRegion({seed}, grid) for seed in seeds]
    
    for i, seed in enumerate(seeds):
        grid.hexes[seed].style = colors[i]
        grid.hexes[seed].label = str(i)
    
    claimed = set(seeds)
    total = grid.nRows * grid.nCols
    arrow_layer = ""
    
    # Grow until everything is claimed
    for round_num in range(30):
        if len(claimed) >= total:
            break
        
        # Collect ALL unclaimed candidates and which regions want them
        candidates = {}  # hex_idx -> [(region_idx, source, dist), ...]
        
        for i, region in enumerate(regions):
            centroid_idx = region.centroid_hex()
            centroid_pos = grid.index_to_hexposition(centroid_idx)
            
            outside = region.outside()
            for candidate in outside.hexes:
                if candidate in claimed:
                    continue
                    
                neighbors = grid.neighborsOf(candidate)
                source = next((n for n in neighbors if n in region.hexes), None)
                
                if source is not None:
                    candidate_pos = grid.index_to_hexposition(candidate)
                    dist = candidate_pos.distance(centroid_pos)
                    
                    if candidate not in candidates:
                        candidates[candidate] = []
                    candidates[candidate].append((i, source, dist))
        
        if not candidates:
            break
        
        # Assign each candidate to closest region (or first if tied)
        for candidate, claimants in candidates.items():
            claimants.sort(key=lambda x: (x[2], x[0]))  # Sort by dist, then region idx for ties
            region_idx, source, dist = claimants[0]
            
            claimed.add(candidate)
            regions[region_idx] = HexRegion(regions[region_idx].hexes | {candidate}, grid)
            grid.hexes[candidate].style = colors[region_idx]
            arrow_layer += grid.arrow(source, candidate, style=arrow_style)
    
        for region in regions:
            paths = region.trace_perimeter(style=perimeter_style)
            boundary_layer += "".join([x.make_windy(iterations=2, offset_factor=0.2).svg() for x in paths])
    
    grid.update()
    grid.builder.adjust("arrows", arrow_layer)
    grid.builder.adjust("boundaries", boundary_layer)
    return grid.builder.show()


In [ ]:
#| export

demo_voronoi_growth_simple()

In [ ]:
#| export
from HexMagic.terrain import Terrain

In [ ]:
#| export
def demoRidge(makeLabels = True):

    mySize = MapSize(240,240)
    myBounds = MapRect(MapCord(0,0), mySize)
    sampleMap =  Terrain(myBounds,radius=20)
    grid = sampleMap.hexGrid

    start = 31
    end = 116

    labelStyle = StyleCSS("labelStyle",fill="purple",stroke="black",stroke_width=1)

    #Need to add styles as we go along
    grid.builder.add_style(labelStyle)

    #itterate through the hexes
    for i in range(len(grid.hexes)):
        if makeLabels:
            grid.hexes[i].label = str(i) 
        grid.hexes[i].labelStyle = labelStyle.name


    start_pos = grid.index_to_hexposition(start)
    end_pos = grid.index_to_hexposition(end)

    arrowLayer = ""
    prev_idx = start_idx = start
    
    # Draw a line between them using cube coordinate interpolation
    path_positions = start_pos.line_to(end_pos)
    
    for i, pos in enumerate(path_positions[1:], 1):  # Skip first (it's the start)
        curr_idx = grid.hexposition_to_index(pos)
        sampleMap.elevations += sampleMap.volcano(center=curr_idx,adjusted=200,
            num_rings=4,variability=0.5,initial_threshold=0.4)

         # Label intermediate hexes
        if curr_idx != end and curr_idx >= 0:
            grid.hexes[curr_idx].label = str(i)
        
        # Draw arrow if valid index
        if curr_idx >= 0:
            arrowLayer += grid.arrow(prev_idx, curr_idx)
            prev_idx = curr_idx

    sampleMap.colorMap()
    grid.update()
    grid.builder.adjust("arrows", arrowLayer)

    return grid.builder.show()


In [ ]:
demoRidge(False)

This is the article I am using for a reference [vornoi](http://www-cs-students.stanford.edu/~amitp/game-programming/polygon-map-generation/).

I am going to refer using the x,y coordinate versions of this as xy coordinates, the array version as np coordinates, and the hexpostion as hex coordinates

1. figure out how to get evenly spaced points xy points in np coorinates
2. itterate through the picked npcoordinates and both randomaly offset using hex (random 2 directions and distance) and eliminated
3. use these to generate voronoi hexregions
4. figure out a certain number of them to be ocean
5. use a distance from coast method to figure out elevations.

right now pompei is 50x50 which I think is a good starting grid size. we will scale down to display.



In [ ]:
#| export


def voronoi_seeds(rows, cols, num_seeds, elimination_rate=0.3, max_offset=3, s=None,debug=False):
    """
    Generate evenly-spaced seeds with randomization for Voronoi diagram.
    
    Args:
        rows: Number of rows in grid
        cols: Number of columns in grid
        num_seeds: Target number of seeds (approximate)
        elimination_rate: Fraction of initial seeds to remove (0-1)
        max_offset: Maximum random offset in each direction
        seed: Random seed for reproducibility
    
    Returns:
        Array of (row, col) coordinates and corresponding 1D indexes
    """
    if s is not None:
        np.random.seed(s)
    

    # Calculate spacing to get approximately num_seeds AFTER elimination
    total_cells = rows * cols
    target_before_elimination = num_seeds / (1 - elimination_rate)
    spacing = int(np.sqrt(total_cells / target_before_elimination))
    spacing = max(1, spacing)

    
    # Create evenly spaced grid
    row_coords = np.arange(spacing//2, rows, spacing)
    col_coords = np.arange(spacing//2, cols, spacing)
    rr, cc = np.meshgrid(row_coords, col_coords)
    seeds = np.column_stack([rr.ravel(), cc.ravel()])
    
    print(f"Initial seeds: {len(seeds)} (spacing={spacing})")
    
    # Elimination step
    keep_mask = np.random.random(len(seeds)) > elimination_rate
    seeds = seeds[keep_mask]
    print(f"After elimination: {len(seeds)}")
    
    # Offset step
    offsets = np.random.uniform(-max_offset, max_offset, size=seeds.shape)
    seeds = seeds + offsets
    
    # Clip to valid range and convert to integers
    seeds[:, 0] = np.clip(seeds[:, 0], 0, rows - 1)
    seeds[:, 1] = np.clip(seeds[:, 1], 0, cols - 1)
    seeds = seeds.astype(int)
    
    # Remove duplicates
    seeds = np.unique(seeds, axis=0)
    print(f"Final unique seeds: {len(seeds)}")
    
    # Convert to 1D indexes
    indexes = seeds[:, 0] * cols + seeds[:, 1]
    
    return seeds, indexes

# Test with 25x25 grid, targeting 6 seeds
coords, indexes = voronoi_seeds(25, 25, num_seeds=6, s=42)

print("\nSeed coordinates (row, col):")
print(coords)
print("\n1D indexes:")
print(indexes)


In [ ]:
#| export
class PlateKind(Enum):
    oceanic = 0        # Ocean/sea
    continental = 1    # Lakes/rivers
    island_arc = 2        # Cold, low precipitation
    mixed = 3        # Hot/cold, very low precipitation
class Plate:
    def __init__(self,region:HexRegion,symbol="♠️",direction=4):
        self.region = region
        self.symbol = symbol
        self.direction = HexPosition.directions()[direction]
        self.kind = PlateKind.oceanic
        self.oceanDistance = 0
        

    def changeDirection(self,direction):
        self.direction = HexPosition.directions()[direction]


    @property
    def hexes(self):
        return self.region.hexes

    @classmethod
    def colors(cls):
        return {
    PlateKind.oceanic: [
        "#2E5266",  # Deep ocean blue
        "#1A4B68",  # Dark blue-gray
        "#2A5674",  # Steel blue
        "#345D7E",  # Slate blue
        "#3E6888",  # Ocean blue
    ],
    PlateKind.continental: [
        "#8B7355",  # Sandy brown
        "#A08968",  # Warm brown
        "#7A6A53",  # Earth brown
        "#9B8B6F",  # Tan brown
        "#6B5D47",  # Dark earth
    ],
    PlateKind.mixed: [
        "#5C6F7A",  # Blue-gray
        "#6A7B85",  # Slate gray
        "#7A8C95",  # Light slate
        "#4D5E69",  # Dark slate
        "#556673",  # Medium slate
    ],
    PlateKind.island_arc: [
    "#6B8E7F",  # Volcanic green-gray
    "#7A9B8C",  # Sage green
    "#5A7A6B",  # Dark moss
    "#8AA599",  # Light seafoam
    "#4F6B5E",  # Deep forest-gray
]

}
    def encode(self) -> str:
        dir_idx = HexPosition.directions().index(self.direction)
        hexes_str = ','.join(str(h) for h in sorted(self.region.hexes))
        return f"{self.symbol}\t{self.kind.value}\t{dir_idx}\t{hexes_str}"

    @classmethod
    def decode(cls, s: str, hex_grid: HexGrid) -> 'Plate':
        parts = s.split('\t')
        symbol = parts[0]
        kind = PlateKind(int(parts[1]))
        direction = int(parts[2])
        hexes = set(int(h) for h in parts[3].split(','))
        region = HexRegion(hexes, hex_grid=hex_grid)
        plate = cls(region, symbol=symbol, direction=direction)
        plate.kind = kind
        return plate



    @classmethod
    def icon(cls, index:int,plates:['Plate'])->str:
        for plate in plates:
            if index in plate.region.hexes:
                return plate.symbol
        return str(index)

    @classmethod
    def voronoi(cls,grid:HexGrid,amount:int=10,sym=["♣️","♦️","♥️","♠️"])->['Plate']:

        seeds = voronoi_seeds(grid.nRows,grid.nCols,amount)[1]

        plates = []
        claimed = set()
        for i, seed in enumerate(seeds):
            icon = str(i)
            if i < len(sym):
                icon = sym[i]
            reg = HexRegion(set([seed]), hex_grid = grid ) # Set of hex indices
            plate = Plate(reg,symbol=icon)
            claimed.add(seed)
            plates.append(plate)

        total = grid.nRows * grid.nCols 

        while len(claimed) < total:
            for plate in plates:
                reg = plate.region.outside()
                adds =[x for x in reg.hexes if x not in claimed]
                for add in adds:
                    claimed.add(add)
                    plate.hexes.add(add)
        return plates

In [ ]:
#| export
def demoVornoi(rings=5,points=10,radius=20):
    grid = sampleGrid(rings, radius=radius, fill="lightgray")
    plates = Plate.voronoi(grid,points)
    styles = StyleCSS.seaborn("hls", len(plates))

    for i, plate in enumerate(plates):
        plateStyle = styles[i]
        grid.builder.add_style(plateStyle)
        for k in plate.hexes:
            grid.hexes[k].style = plateStyle

    grid.update()
    return grid

In [ ]:
#| export
vern = demoVornoi(rings=15,points=80,radius=10)
vern.builder.show()

In [ ]:
#| export
def demoOutlineVornoi(rings=5,points=10,radius=20):
    grid = sampleGrid(rings, radius=radius, fill="lightgray")
    plates = Plate.voronoi(grid,points)
    styles = StyleCSS.seaborn("hls", len(plates))

    plateLayer = ""

    for i, plate in enumerate(plates):
        for path in plate.region.trace_perimeter():
            plateStyle = styles[i]
            plateStyle.properties["stroke"] = plateStyle.properties["fill"]
            plateStyle.properties["fill"] = "none"
            plateStyle.properties["stroke_width"] = 2
            grid.builder.add_style(plateStyle)

            path.style = plateStyle
            
            plateLayer += path.drawPloy()

    #grid.update()
    for x in grid.builder.layers:
        print(x.name)
        grid.builder.adjust(x.name,"")
    grid.builder.adjust("plates", plateLayer)
    return grid

In [ ]:
#| export
dOV = demoOutlineVornoi(rings=15,points=80,radius=10)
dOV.builder.show()

In [ ]:
??HexPosition.distance

we are have the first step of good voronoi. we now need to mark some of these as oceanic and then do a distance for each regions centroid to see how far away it is.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Looking at your code, I can help you implement the next steps. Here's how we can mark some plates as oceanic and calculate distances from centroids:

```python
def mark_oceanic_plates(plates, ocean_fraction=0.4, seed=None):
    """
    Randomly mark some plates as oceanic.
    
    Args:
        plates: List of Plate objects
        ocean_fraction: Fraction of plates to mark as oceanic (0-1)
        seed: Random seed for reproducibility
    
    Returns:
        Modified plates list
    """
    if seed is not None:
        np.random.seed(seed)
    
    num_ocean = int(len(plates) * ocean_fraction)
    ocean_indices = np.random.choice(len(plates), size=num_ocean, replace=False)
    
    for i in ocean_indices:
        plates[i].kind = PlateKind.oceanic
    
    # Mark the rest as continental
    for i, plate in enumerate(plates):
        if i not in ocean_indices:
            plate.kind = PlateKind.continental
    
    return plates


def calculate_distances_from_ocean(plates, grid):
    """
    Calculate distance from each hex to nearest oceanic plate centroid.
    
    Args:
        plates: List of Plate objects
        grid: HexGrid object
    
    Returns:
        Dictionary mapping hex_index -> distance_to_nearest_ocean
    """
    # Get oceanic plate centroids
    ocean_centroids = []
    for plate in plates:
        if plate.kind == PlateKind.oceanic:
            centroid_idx = plate.region.centroid_hex()
            centroid_pos = grid.index_to_hexposition(centroid_idx)
            ocean_centroids.append(centroid_pos)
    
    if not ocean_centroids:
        print("Warning: No oceanic plates found!")
        return {}
    
    # Calculate distance for each hex
    distances = {}
    for idx in range(len(grid.hexes)):
        hex_pos = grid.index_to_hexposition(idx)
        
        # Find minimum distance to any ocean centroid
        min_dist = min(hex_pos.distance(ocean_pos) for ocean_pos in ocean_centroids)
        distances[idx] = min_dist
    
    return distances
```

Here's a demo function that shows it all working together:

```python
def demoOceanicDistance(rings=5, points=10, radius=20, ocean_fraction=0.4):
    grid = sampleGrid(rings, radius=radius, fill="lightgray")
    plates = Plate.voronoi(grid, points)
    
    # Mark some as oceanic
    plates = mark_oceanic_plates(plates, ocean_fraction=ocean_fraction, seed=42)
    
    # Calculate distances
    distances = calculate_distances_from_ocean(plates, grid)
    
    # Create styles for oceanic vs continental
    ocean_colors = Plate.colors()[PlateKind.oceanic]
    continent_colors = Plate.colors()[PlateKind.continental]
    
    # Color plates by type
    for plate in plates:
        if plate.kind == PlateKind.oceanic:
            color = ocean_colors[0]
        else:
            color = continent_colors[0]
        
        style = StyleCSS(f"plate_{plate.symbol}", fill=color, stroke="#333", stroke_width=1)
        grid.builder.add_style(style)
        
        for hex_idx in plate.hexes:
            grid.hexes[hex_idx].style = style
            # Show distance as label
            grid.hexes[hex_idx].label = str(distances.get(hex_idx, '?'))
    
    grid.update()
    
    # Print some stats
    print(f"Total plates: {len(plates)}")
    print(f"Oceanic plates: {sum(1 for p in plates if p.kind == PlateKind.oceanic)}")
    print(f"Max distance from ocean: {max(distances.values())}")
    print(f"Min distance from ocean: {min(distances.values())}")
    
    return grid
```

This gives you:
1. Random assignment of oceanic vs continental plates
2. Distance calculation from each hex to the nearest oceanic plate centroid
3. Visual display showing the distances

You can then use these distances for elevation generation - typically closer to ocean = lower elevation.

In [ ]:
#| export
def mark_oceanic_plates(plates, ocean_fraction=0.4, seed=None):
    """
    Randomly mark some plates as oceanic.
    
    Args:
        plates: List of Plate objects
        ocean_fraction: Fraction of plates to mark as oceanic (0-1)
        seed: Random seed for reproducibility
    
    Returns:
        Modified plates list
    """
    if seed is not None:
        np.random.seed(seed)
    
    num_ocean = int(len(plates) * ocean_fraction)
    ocean_indices = np.random.choice(len(plates), size=num_ocean, replace=False)
    
    for i in ocean_indices:
        plates[i].kind = PlateKind.oceanic
    
    # Mark the rest as continental
    for i, plate in enumerate(plates):
        if i not in ocean_indices:
            plate.kind = PlateKind.continental
    
    return plates




In [ ]:
#| export
def calculate_distances_from_ocean(plates, grid):
    """
    Calculate distance from each plate to nearest oceanic plate centroid.
    """
  
    # Get oceanic plate centroids
    ocean_centroids = []
    for plate in plates:
        if plate.kind == PlateKind.oceanic:
            centroid_idx = plate.region.centroid_hex()
            centroid_pos = grid.index_to_hexposition(centroid_idx)
            ocean_centroids.append(centroid_pos)
    
    if not ocean_centroids:
        print("Warning: No oceanic plates found!")
        return 

    for plate in plates:
        if plate.kind != PlateKind.oceanic:
            centroid_idx = plate.region.centroid_hex()
            hex_pos = grid.index_to_hexposition(centroid_idx)
            plate.oceanDistance = min(hex_pos.distance(ocean_pos) for ocean_pos in ocean_centroids)
   


In [ ]:
#| export
def demoOceanicDistance(rings=5, points=10, radius=20, ocean_fraction=0.4):
    grid = sampleGrid(rings, radius=radius, fill="lightgray")
    plates = Plate.voronoi(grid, points)
    
    # Mark some as oceanic
    plates = mark_oceanic_plates(plates, ocean_fraction=ocean_fraction, seed=42)
    
    # Calculate distances
    calculate_distances_from_ocean(plates, grid)

    levels = max([x.oceanDistance for x in plates])
    styles = StyleCSS.seaborn("terrain", levels + 1)
 
    
    # Color plates by type
    for plate in plates:
        plateStyle = styles[plate.oceanDistance]
        grid.builder.add_style(plateStyle)
        for k in plate.hexes:
            grid.hexes[k].style = plateStyle


    
    grid.update()
    
    # Print some stats
    print(f"Total plates: {len(plates)}")
    print(f"Oceanic plates: {sum(1 for p in plates if p.kind == PlateKind.oceanic)}")

    
    return grid


In [ ]:
#| export
dOD = demoOceanicDistance(rings=25,points=300,radius=5, ocean_fraction=0.2)
dOD.builder.show()

For the final step we need to use this with Terrain and have it generate random elevations for a new terrain

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great! Let's integrate the Voronoi plates with the Terrain class to generate random elevations. Here's how we can do it:

```python
def generate_plate_terrain(bounds, radius=20, num_plates=10, ocean_fraction=0.4, seed=None):
    """
    Generate terrain based on tectonic plates using Voronoi regions.
    
    Args:
        bounds: MapRect defining the terrain bounds
        radius: Hex radius
        num_plates: Number of tectonic plates
        ocean_fraction: Fraction of plates that are oceanic
        seed: Random seed for reproducibility
    
    Returns:
        Terrain object with plate-based elevations
    """
    if seed is not None:
        np.random.seed(seed)
    
    # Create terrain and grid
    terrain = Terrain(bounds, radius=radius)
    grid = terrain.hexGrid
    
    # Generate Voronoi plates
    plates = Plate.voronoi(grid, num_plates)
    
    # Mark some as oceanic
    plates = mark_oceanic_plates(plates, ocean_fraction=ocean_fraction, seed=seed)
    
    # Calculate distances from ocean
    calculate_distances_from_ocean(plates, grid)
    
    # Generate elevations based on plate properties
    elevations = np.zeros(len(grid.hexes))
    
    for plate in plates:
        if plate.kind == PlateKind.oceanic:
            # Ocean plates are low elevation (negative)
            base_elevation = -100
            variation = 30
        else:
            # Continental plates elevation based on distance from ocean
            # Farther from ocean = higher elevation
            base_elevation = 50 + (plate.oceanDistance * 20)
            variation = 40
        
        # Add random variation to each hex in the plate
        for hex_idx in plate.hexes:
            elevation = base_elevation + np.random.uniform(-variation, variation)
            elevations[hex_idx] = elevation
    
    # Smooth the elevations at plate boundaries
    smoothed = elevations.copy()
    for i in range(len(grid.hexes)):
        neighbors = grid.neighborsOf(i)
        if neighbors:
            # Average with neighbors for smoothing
            neighbor_vals = [elevations[n] for n in neighbors if n < len(elevations)]
            if neighbor_vals:
                smoothed[i] = 0.7 * elevations[i] + 0.3 * np.mean(neighbor_vals)
    
    terrain.elevations = smoothed
    terrain.colorMap()
    
    return terrain, plates


# Demo function
def demoPlateGeneration(map_size=240, radius=20, num_plates=15, ocean_fraction=0.3):
    """Demo the plate-based terrain generation"""
    bounds = MapRect(MapCord(0, 0), MapSize(map_size, map_size))
    
    terrain, plates = generate_plate_terrain(
        bounds, 
        radius=radius, 
        num_plates=num_plates,
        ocean_fraction=ocean_fraction,
        seed=42
    )
    
    grid = terrain.hexGrid
    
    # Add labels showing plate info
    labelStyle = StyleCSS("labelStyle", fill="white", stroke="black", stroke_width=1)
    grid.builder.add_style(labelStyle)
    
    # Optionally show plate boundaries
    boundaryLayer = ""
    boundary_style = StyleCSS("boundary", fill="none", stroke="red", stroke_width=2, opacity=0.5)
    grid.builder.add_style(boundary_style)
    
    for plate in plates:
        paths = plate.region.trace_perimeter(style=boundary_style)
        for path in paths:
            boundaryLayer += path.svg()
    
    grid.builder.adjust("boundaries", boundaryLayer)
    grid.update()
    
    print(f"Generated terrain with {len(plates)} plates")
    print(f"Oceanic plates: {sum(1 for p in plates if p.kind == PlateKind.oceanic)}")
    print(f"Continental plates: {sum(1 for p in plates if p.kind == PlateKind.continental)}")
    print(f"Elevation range: {terrain.elevations.min():.1f} to {terrain.elevations.max():.1f}")
    
    return terrain
```

Now you can test it:

```python
terrain = demoPlateGeneration(map_size=240, radius=10, num_plates=20, ocean_fraction=0.3)
terrain.hexGrid.builder.show()
```

This approach:

1. **Creates Voronoi plates** using your existing `Plate.voronoi()` method
2. **Marks oceanic vs continental plates** randomly based on the ocean fraction
3. **Calculates distances** from oceanic plates for continental plates
4. **Generates elevations**:
   - Oceanic plates get negative elevations (underwater)
   - Continental plates get elevations based on their distance from ocean (farther = higher)
   - Adds random variation within each plate
5. **Smooths boundaries** between plates to avoid sharp transitions
6. **Colors the map** using your existing terrain color scheme

You can adjust parameters like `num_plates`, `ocean_fraction`, and the elevation formulas to get different terrain characteristics!

In [ ]:


from HexMagic.terrain import Terrain



In [ ]:
#| export
def generate_plate_terrain(bounds, radius=20, num_plates=10, ocean_fraction=0.4, seed=None):
    """
    Generate terrain based on tectonic plates using Voronoi regions.
    
    Args:
        bounds: MapRect defining the terrain bounds
        radius: Hex radius
        num_plates: Number of tectonic plates
        ocean_fraction: Fraction of plates that are oceanic
        seed: Random seed for reproducibility
    
    Returns:
        Terrain object with plate-based elevations
    """
    if seed is not None:
        np.random.seed(seed)
    
    # Create terrain and grid
    terrain = Terrain(bounds, radius=radius)
    grid = terrain.hexGrid
    
    # Generate Voronoi plates
    plates = Plate.voronoi(grid, num_plates)
    
    # Mark some as oceanic
    plates = mark_oceanic_plates(plates, ocean_fraction=ocean_fraction, seed=seed)
    
    # Calculate distances from ocean
    calculate_distances_from_ocean(plates, grid)
    
    # Generate elevations based on plate properties
    elevations = np.zeros(len(grid.hexes))
    
    for plate in plates:
        if plate.kind == PlateKind.oceanic:
            # Ocean plates are low elevation (negative)
            base_elevation = -100
            variation = 30
        else:
            # Continental plates elevation based on distance from ocean
            # Farther from ocean = higher elevation
            base_elevation = 50 + (plate.oceanDistance * 20)
            variation = 40
        
        # Add random variation to each hex in the plate
        for hex_idx in plate.hexes:
            elevation = base_elevation + np.random.uniform(-variation, variation)
            elevations[hex_idx] = elevation
    
    # Smooth the elevations at plate boundaries
    smoothed = elevations.copy()
    for i in range(len(grid.hexes)):
        neighbors = grid.neighborsOf(i)
        if neighbors:
            # Average with neighbors for smoothing
            neighbor_vals = [elevations[n] for n in neighbors if n < len(elevations)]
            if neighbor_vals:
                smoothed[i] = 0.7 * elevations[i] + 0.3 * np.mean(neighbor_vals)
    
    terrain.elevations = smoothed
    terrain.colorMap()
    
    return terrain, plates



Terrain.fromSeeds = generate_plate_terrain

In [ ]:
#| export
def demoTerr():

    mySize = MapSize(480,480)
    myBounds = MapRect(MapCord(0,0), mySize)
    sampleMap, plates =  Terrain.fromSeeds(myBounds,radius=15)

    sampleMap.colorMap()
    sampleMap.hexGrid.update()

    return sampleMap.hexGrid.builder.show()

In [ ]:
#| export
demoTerr()